# Channel Break Out Strategy

## Import Data

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [2]:
df = pd.read_csv("EURUSD_Candlestick_1_D_BID_05.05.2003-28.10.2023.csv")

## Detect Pivots/Fractals

In [3]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0
    

## Detect Price Channel

In [4]:
def collect_channel(candle, backcandles, window, parallel):
    localdf = df[candle-backcandles-window:candle-window]
    localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    highs = localdf[localdf['isPivot']==1].High.values[-3:]
    idxhighs = localdf[localdf['isPivot']==1].High.index[-3:]
    lows = localdf[localdf['isPivot']==2].Low.values[-3:]
    idxlows = localdf[localdf['isPivot']==2].Low.index[-3:]
    total_length = len(lows) + len(highs)
    if len(lows)>=2 and len(highs)>=2 and total_length >= 5:
        sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)
        
        if not (parallel>0) or abs( (sl_lows-sl_highs)/(sl_highs+sl_lows)/2 ) < parallel:
             return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
    return(0,0,0,0,0,0)

In [5]:
backcandles = 40
window = 4

In [6]:
def pointpos(x):
    if x['isPivot']==2:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [7]:
candle = 422

dfpl = df[candle-backcandles-window-5:candle+200]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle, backcandles, window, parallel=0.1)
print(r_sq_l, r_sq_h)
if sl_highs and sl_lows:
    x = np.array(range(candle-backcandles-window, candle-window+1))
    fig.add_trace(go.Scatter(x=x, y=sl_lows*x + interc_lows, mode='lines', name='lower slope'))
    fig.add_trace(go.Scatter(x=x, y=sl_highs*x + interc_highs, mode='lines', name='max slope'))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1493752777.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0 0


## Detect Break Out

In [8]:
def isBreakOut(candle, backcandles, window, parallel):
    if (candle-backcandles-window)<0:
        return 0
    
    sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle, 
                                                                                   backcandles, 
                                                                                   window, 
                                                                                   parallel=parallel)
    
    prev_idx = candle-1
    prev_high = df.iloc[candle-1].High
    prev_low = df.iloc[candle-1].Low
    prev_close = df.iloc[candle-1].Close
    
    curr_idx = candle
    curr_high = df.iloc[candle].High
    curr_low = df.iloc[candle].Low
    curr_close = df.iloc[candle].Close
    curr_open = df.iloc[candle].Open

    if ( #prev_high > (sl_lows*prev_idx + interc_lows) and
        #prev_close < (sl_lows*prev_idx + interc_lows) and
        curr_open < (sl_lows*curr_idx + interc_lows) and
        curr_close < (sl_lows*prev_idx + interc_lows) and 
        r_sq_l > 0.9 and r_sq_h > 0.9):
        return 1
    
    elif ( #prev_low < (sl_highs*prev_idx + interc_highs) and
        #prev_close > (sl_highs*prev_idx + interc_highs) and
        curr_open > (sl_highs*curr_idx + interc_highs) and
        curr_close > (sl_highs*prev_idx + interc_highs) and 
        r_sq_h > 0.9 and r_sq_l > 0.9):
        return 2
    
    else:
        return 0

## Generate signal for all dataframe

In [9]:
from tqdm import tqdm
backcandles = 40
window = 5

df["isBreakOut"] = [isBreakOut(candle, backcandles, window, parallel=0.1) for candle in tqdm(df.index)]

  0%|          | 0/6410 [00:00<?, ?it/s]

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1493752777.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  1%|          | 46/6410 [00:00<00:15, 422.02it/s]C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1493752777.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1493752777.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [10]:
df[df["isBreakOut"]!=0][:30]

,Gmt time,Open,High,Low,Close,Volume,isPivot,pointpos,isBreakOut
109,10.09.2003 00:00:00.000,1.12169,1.12347,1.11608,1.11811,1.364767e+06,0,NaN,2
110,11.09.2003 00:00:00.000,1.11792,1.12695,1.11537,1.11831,1.364461e+06,0,NaN,2
111,12.09.2003 00:00:00.000,1.11816,1.13208,1.11329,1.12877,1.275472e+06,1,1.13308,2
112,14.09.2003 00:00:00.000,1.12675,1.12779,1.12502,1.12748,9.064990e+04,0,NaN,2
113,15.09.2003 00:00:00.000,1.12744,1.13082,1.12401,1.12766,1.350359e+06,0,NaN,2
114,16.09.2003 00:00:00.000,1.12765,1.13165,1.11523,1.11632,1.345839e+06,0,NaN,2
115,17.09.2003 00:00:00.000,1.11625,1.12989,1.11366,1.12750,1.340350e+06,0,NaN,2
116,18.09.2003 00:00:00.000,1.12750,1.13414,1.12246,1.12482,1.338139e+06,0,NaN,2
564,22.02.2005 00:00:00.000,1.30608,1.32670,1.30504,1.32569,1.332764e+06,0,NaN,2
565,23.02.2005 00:00:00.000,1.32545,1.32731,1.31843,1.31992,1.332008e+06,0,NaN,2


In [11]:
candle = 564

def breakpointpos(x):
    if x['isBreakOut']==2:
        return x['Low']-3e-3
    elif x['isBreakOut']==1:
        return x['High']+3e-3
    else:
        return np.nan

dfpl = df[candle-backcandles-window-5:candle+100]
dfpl["breakpointpos"] = dfpl.apply(lambda row: breakpointpos(row), axis=1)

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

fig.add_scatter(x=dfpl.index, y=dfpl['breakpointpos'], mode="markers",
                marker=dict(size=8, color="Black"), marker_symbol="hexagram",
                name="pivot")

sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle, backcandles, window, parallel=0)
print(r_sq_l, r_sq_h)
if sl_highs and sl_lows:
    x = np.array(range(candle-backcandles-window, candle-window+1))
    fig.add_trace(go.Scatter(x=x, y=sl_lows*x + interc_lows, mode='lines', name='lower slope'))
    fig.add_trace(go.Scatter(x=x, y=sl_highs*x + interc_highs, mode='lines', name='max slope'))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1472306223.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.9892817634573432 1.0


C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\1493752777.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
import pandas_ta as ta
df['RSI'] = ta.rsi(df['Close'], length=4)
#df.set_index("Gmt time", inplace=True)
# df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S.%f').floor('S')
# df

In [13]:
def SIGNAL():
    return df.isBreakOut

In [14]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 10000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5
        perc = 0.03
        

        if self.signal!=0 and len(self.trades)==0 and self.data.isBreakOut==2:
            sl = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]+sldiff*TPSLRatio
            self.buy(sl=sl, tp=tp, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.isBreakOut==1:         
            sl = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]-sldiff*TPSLRatio
            self.sell(sl=sl, tp=tp, size=self.mysize)

bt = Backtest(df, MyStrat, cash=10000, margin=1/5)
stat = bt.run()
stat

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\2337034015.py:28: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    6409.0
Duration                               6409.0
Exposure Time [%]                    25.50702
Equity Final [$]                   11909.3505
Equity Peak [$]                     13443.717
Return [%]                          19.093505
Buy & Hold Return [%]               -6.469838
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.088632
Avg. Drawdown [%]                   -1.392685
Max. Drawdown Duration                 1231.0
Avg. Drawdown Duration              65.818182
# Trades                                 26.0
Win Rate [%]                        46.153846
Best Trade [%]                       4.702261
Worst Trade [%]                     -3.002701
Avg. Trade [%]                    

In [15]:
bt.plot()

Row(id='1455', ...)

In [16]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 10000
    ordertime=[]
    def init(self):
        super().init()

    def next(self):
        super().next()
        TPSLRatio = 1.8
        perc = 0.01
        slperc = 0.03

        #Close trades if RSI is above 70 for long positions and below 30 for short positions
        for trade in self.trades:
            if trade.is_long and self.data.RSI[-1] > 95:
                trade.close()
            elif trade.is_short and self.data.RSI[-1] < 5:
                trade.close()

        if len(self.orders)==1:
            if (self.data.index[-1]-self.ordertime[0])>2:
                self.orders[0].cancel()
                self.ordertime.pop(0)
        
        if len(self.orders)==0:
            self.ordertime.clear()

        if len(self.trades)==0 and self.data.isBreakOut[-1]==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
                
            order_price = self.data.Close[-1] - self.data.Close[-1]*perc
            sl = order_price-order_price*slperc
            sldiff = abs(sl-order_price)
            tp = order_price+sldiff*TPSLRatio
            self.buy(limit=order_price, sl=sl, tp=tp, size=self.mysize)
            self.ordertime.append(self.data.index[-1])

        elif len(self.trades)==0 and self.data.isBreakOut[-1]==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                #print("signal deleting orders", self.orders)
                self.orders[0].cancel()
                self.ordertime.pop(0)
                
            order_price = self.data.Close[-1] + self.data.Close[-1]*perc
            sl = order_price+order_price*slperc
            sldiff = abs(sl-order_price)
            tp = order_price-sldiff*TPSLRatio
            self.sell(limit=order_price, sl=sl, tp=tp, size=self.mysize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(df, MyStrat, cash=10000, margin=1/5)
stat = bt.run()
stat

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_1420\3862964547.py:58: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    6409.0
Duration                               6409.0
Exposure Time [%]                   12.418097
Equity Final [$]                  13197.95943
Equity Peak [$]                   13981.77189
Return [%]                          31.979594
Buy & Hold Return [%]               -6.469838
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.288117
Avg. Drawdown [%]                   -1.138686
Max. Drawdown Duration                 1137.0
Avg. Drawdown Duration              59.804348
# Trades                                 13.0
Win Rate [%]                        61.538462
Best Trade [%]                            5.4
Worst Trade [%]                          -3.0
Avg. Trade [%]                    

In [17]:
bt.plot()

Row(id='2559', ...)

In [18]:
df[df["isBreakOut"]!=0]

,Gmt time,Open,High,Low,Close,Volume,isPivot,pointpos,isBreakOut,RSI
109,10.09.2003 00:00:00.000,1.12169,1.12347,1.11608,1.11811,1.364767e+06,0,NaN,2,71.005234
110,11.09.2003 00:00:00.000,1.11792,1.12695,1.11537,1.11831,1.364461e+06,0,NaN,2,71.266758
111,12.09.2003 00:00:00.000,1.11816,1.13208,1.11329,1.12877,1.275472e+06,1,1.13308,2,82.361128
112,14.09.2003 00:00:00.000,1.12675,1.12779,1.12502,1.12748,9.064990e+04,0,NaN,2,77.444096
113,15.09.2003 00:00:00.000,1.12744,1.13082,1.12401,1.12766,1.350359e+06,0,NaN,2,77.691875
...,...,...,...,...,...,...,...,...,...,...
6396,13.10.2023 00:00:00.000,1.05358,1.05584,1.04954,1.05073,3.012774e+05,2,1.04854,2,28.777774
6397,15.10.2023 00:00:00.000,1.05083,1.05253,1.05025,1.05220,1.266406e+04,0,NaN,2,37.049884
6398,16.10.2023 00:00:00.000,1.05221,1.05627,1.05156,1.05543,2.201124e+05,0,NaN,2,53.031794
6399,17.10.2023 00:00:00.000,1.05543,1.05948,1.05328,1.05727,2.812887e+05,0,NaN,2,60.624730
